In [1]:
import cv2
import json
import os
import os.path as osp
import numpy as np
from pathlib import Path
from tqdm import tqdm
from time import time

import pandas as pd
from sklearn.metrics import confusion_matrix

from core import ObjectDetector, CornerPredictor

In [2]:
#dt_vehicle = ObjectDetector('yolov4-default')
dt_plate   = ObjectDetector('yolov4-plate_type')
#dt_number  = ObjectDetector('yolov4-plate_number')
#dt_corner  = CornerPredictor('corner_prediction')

<yolov4-plate_type> model loaded.


In [3]:
def start(img, pts):
    plate = ''
    s_time = time()

    img_vehicle = dt_plate.loadImage(img)
    bbox2, obj2 = dt_plate.runDetection(mode='plate', multi_res=True)
    for i, box2 in enumerate(bbox2):
        if box2[0, 0] <= pts[0, 0] and box2[0, 1] <= pts[0, 1] and\
        box2[1, 0] >= pts[1, 0] and box2[1, 1] >= pts[1, 1]:
            plate = obj2[i]
            break

    t_time = time() - s_time

    return t_time, plate, img_vehicle

def error_log(p, label, predict, write=False, img=None):
    log = {'path': p, 'label': label, 'predict': predict}
    
    if write:
        dir = 'error_log/plate_error_ourDataset'
        file = osp.basename(p).split('-')[0]
        if not osp.exists(dir): os.mkdir(dir)
        cv2.imwrite(osp.join(dir, file), cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    return log

In [4]:
path_file = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\training_data\plate_type_detection\file_path.json'
with open(path_file, 'r') as f:
    test_path = json.load(f)['valid']

test_path = [osp.join(r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection', p).replace('json', 'jpg') for p in test_path]
classes = {'白底黑字': 0, '白底紅字': 1, '電動車': 2, '其他': 3}

print('total:', len(test_path))

total: 3496


In [5]:
n_test = len(test_path)
label, predict = [], []
plate_error, no_detection = [], []
total_time = 0

acc_list = [n_test, 0]

for i in tqdm(range(n_test), unit=' images', ncols=100):
    p = test_path[i]
    img = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB)
    
    with open(p.replace('jpg', 'json'), 'r', encoding='utf-8') as f:
        data = json.load(f)['shapes'][0]
    l_type = data['label'] # plate type
    pts = np.array(data['points']) # plate points 

    offset_x, offset_y = int(0.1*(pts[1, 0]-pts[0, 0])), int(0.1*(pts[1, 1]-pts[0, 1]))
    pts[0, 0], pts[0, 1] = pts[0, 0]+offset_x, pts[0, 1]+offset_y
    pts[1, 0], pts[1, 1] = pts[1, 0]-offset_x, pts[1, 1]-offset_y
    
    if i == 0:
        _, _, _ = start(img, pts) # skip initial time
    t_time, d_type, img_vehicle = start(img, pts)
    total_time += t_time

    if d_type != '':
        label.append(classes[l_type])
        predict.append(classes[d_type])

    fn = osp.basename(p)
    if d_type == l_type:
        acc_list[1] += 1
    elif d_type == '':
        no_detection.append(error_log(fn, l_type, d_type, True, img_vehicle))
    else: 
        plate_error.append(error_log(fn, l_type, d_type, True, img_vehicle))

print('Average time: {:.3f}s'.format(total_time/n_test))
print('Overall: total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[0], acc_list[1], acc_list[1]/acc_list[0]*100))
print('Detection rate: {:.2f}%'.format((acc_list[0]-len(no_detection))/acc_list[0]*100))
print('No detection:', len(no_detection))
print('Plate error:', len(plate_error))

log_dict = {'no_detection': no_detection, 'plate_error': plate_error}
with open('error_log/plate_error_ourDataset.json', 'w', encoding='utf-8') as f:
    json.dump(log_dict, f, indent=2, ensure_ascii=False)

100%|██████████████████████████████████████████████████████| 3496/3496 [02:37<00:00, 22.23 images/s]

Average time: 0.037s
Overall: total=3496, correct=3426, accuracy=98.00%
Detection rate: 98.28%
No detection: 60
Plate error: 10


In [6]:
def calc_score(cfmat):
    cls_num = cfmat.shape[0]
    all_num = np.sum(cfmat)
    cls_score = []

    for i in range(cls_num):
        TP = cfmat[i, i]
        FN = np.sum(cfmat[i, :]) - TP
        FP = np.sum(cfmat[:, i]) - TP
        #TN = all_num - FN - FP + TP
        
        TPR = TP / (TP+FN) if TP != 0 else 0 # recall
        PPV = TP / (TP+FP) if TP != 0 else 0 # precision
        F1 = 2 * (PPV*TPR) / (PPV+TPR) if 0 not in [PPV, TPR] else 0
        
        cls_score.append({'TP': TP,
                        'FN': FN,
                        'FP': FP,
                        'TPR': round(TPR, 4),
                        'PPV': round(PPV, 4),
                        'F1': round(F1, 4)})

    return cls_score

In [7]:
confusion_mat = confusion_matrix(label, predict, labels=[i for i in range(4)])
print(confusion_mat) # row = label, column = predict

score = calc_score(confusion_mat)
pd.DataFrame(score)

[[3169    4    0    0]
 [   1  235    0    1]
 [   0    0   14    0]
 [   3    1    0    8]]


,TP,FN,FP,TPR,PPV,F1
0,3169,4,4,0.9987,0.9987,0.9987
1,235,2,5,0.9916,0.9792,0.9853
2,14,0,0,1.0000,1.0000,1.0000
3,8,4,1,0.6667,0.8889,0.7619
